# Problem 3
we mostly use Problem 2 decision variable, constrains, obj function and add/remove some
## add parameters
* LostSaleCost_i be the lost sale const of product $i$
## add decision varaible
* Let $y_{ij}$ be inventory level of product $i$ in month $j$
* Let $yBin_{ij}$, $wBin_{ij}$ be helper binary variable for $y_{ij}$, $w_{ij}$, this garuantee $max\{0, Expr_{ij}\} = y_{ij}$, $max\{0, -Expr_{ij}\} = w_{ij}$
* Let $w_{ij}$ be the amount of lost sale of product $i$ in month $j$
## add constrains
* $0 \le y_{ij} \quad \forall i,\ \forall j$
* $0 \le w_{ij} \quad \forall i,\ \forall j$
* $ExprY_{ij} \le y_{ij} \quad \forall i,\ \forall j$
* $-ExprY_{ij} \le w_{ij} \quad \forall i,\ \forall j$
* $x_{max} \times yBin_{ij} + ExprY_{ij} \ge y_{ij}$
* $x_{max} \times (1-yBin_{ij}) + 0 \ge y_{ij}$
* $x_{max} \times wBin_{ij} + 0 \ge w_{ij}$
* $x_{max} \times (1-wBin_{ij}) - ExprY_{ij} \ge w_{ij}$
## Ultility Expression $ExprY_{ij}$
* $j=0:\ ExprY_{i0} = I_i - D_{i0}$
* $j=1:\ ExprY_{i1} = y_{i0} + x_{i00} + Transit_{i1} - D_{i1}$
* $j=2:\ ExprY_{i2} = y_{i1} + x_{i10} + x_{i01} + Transit_{i2} - D_{i2}$
* $j \ge 3:\ ExprY_{ij} = y_{i(j-1)} + x_{i(j-1)0} + x_{i(j-2)1} + x_{i(j-3)2} - D_{ij}$
## add objective value
* Inventroy Cost: $\sum_{i=0}^N[(\sum_{j=0}^M y_{ij}) HoldCost_i]$
* Lost Sale Cost: $\sum_{i=0}^N[(\sum_{j=0}^M w_{ij}) LostSaleCost_i]$

In [1]:
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

In [2]:
## setting parameter
N, M, K = 10, 6, 3
D_ij = pd.read_excel('data.xlsx', sheet_name='Demand', index_col='Product').to_numpy()
I_i = pd.read_excel('data.xlsx', sheet_name='Initial inventory', index_col='Product').to_numpy().squeeze()
BuyCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Purchasing cost'].to_numpy().squeeze()
HoldCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Holding'].to_numpy().squeeze()
Transit_ij = pd.read_excel('data.xlsx', sheet_name='In-transit', index_col='Product').to_numpy().squeeze()
FixedShipCost_k = (100, 80, 50)
VarShipCost_ik = pd.read_excel('data.xlsx', sheet_name='Shipping cost', index_col='Product').to_numpy().squeeze()
x_max = sum([sum(i) for i in D_ij]) + sum(I_i)
ContainerCapacity = 30
ContainerCost = 2750
CBM_i = pd.read_excel('data.xlsx', sheet_name='Size', index_col='Product').to_numpy().squeeze()
LostSaleCost_i = pd.read_excel('data.xlsx', sheet_name='Shortage', index_col='Product')['Lost sales'].to_numpy()

In [3]:
m = gb.Model("Problem3Model")

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [4]:
#add decision variable
x = m.addVars(N, M, K, vtype=GRB.INTEGER, name='x_ijk')
z = m.addVars(M, K, vtype=GRB.BINARY, name='z_jk')
ContainerCnt = m.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt_j')
y = m.addVars(N, M, vtype=GRB.CONTINUOUS, name='y_ij')
yBin = m.addVars(N, M, vtype=GRB.BINARY, name='yBin_ij')
w = m.addVars(N, M, vtype=GRB.CONTINUOUS, name='w_ij')
wBin = m.addVars(N, M, vtype=GRB.BINARY, name='wBin_ij')

In [5]:
#add ExprZ_j, ExprY_ij
ExprY_ij = [
    [
        gb.LinExpr(I_i[i] - D_ij[i][0]),
        gb.LinExpr(
            y[i,0] + x[i,0,0] + Transit_ij[i][1] - D_ij[i][1]
        ),
        gb.LinExpr(
            y[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - D_ij[i][2]
        )
    ] +
    [
        gb.LinExpr(y[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - D_ij[i][j])
    for j in range(3, M)]
for i in range(N)]

ExprZ_j = [
    quicksum(x[i,j,2]*CBM_i[i] for i in range(N))
for j in range(0, M)]

In [6]:
#add to objctive function
purchaseCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for k in range(0, K))
        for j in range(0, M))
    for i in range(0, N))
)

FixedCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            z[j,k] * FixedShipCost_k[k]
        for k in range(0, K))
    for j in range(0, M))
)

VariableCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * VarShipCost_ik[i][k]
            for k in range(K))
        for i in range(N))
    for j in range(M))
)

InventoryCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            y[i,j]
        for j in range(0, M)) * HoldCost_i[i]
    for i in range(0, N))
)

allContainerCostExpr =  gb.LinExpr(
    quicksum(ContainerCnt[j] * ContainerCost for j in range(M))
)

AllLostSaleCostExpr = gb.LinExpr(
    quicksum(
        quicksum(w[i,j] for j in range(M)) * LostSaleCost_i[i]
    for i in range(N))
)

m.setObjective(
    purchaseCostExpr + FixedCostExpr + VariableCostExpr + InventoryCostExpr + allContainerCostExpr + AllLostSaleCostExpr
)

In [7]:
#Constrains

_ = m.addConstrs(
    quicksum(x[i,j,k] for i in range(0, N)) / x_max <= z[j,k]
for j in range(0, M)
for k in range(0, K)
)
_ = m.addConstrs(
     ExprZ_j[j] / ContainerCapacity <= ContainerCnt[j]
 for j in range(M))

_ = m.addConstrs(
    ExprY_ij[i][j] <= y[i,j]
    for i in range(N)
    for j in range(M)
)

_ = m.addConstrs(
    x_max * yBin[i,j] + ExprY_ij[i][j] >= y[i,j]
    for i in range(N)
    for j in range(M)
)

_ = m.addConstrs(
    x_max*(1-yBin[i,j]) + 0 >= y[i,j]
    for i in range(N)
    for j in range(M)
)

_ = m.addConstrs(
    -ExprY_ij[i][j] <= w[i,j]
    for i in range(N)
    for j in range(M)
)

_ = m.addConstrs(
    x_max * wBin[i,j] + 0 >= w[i,j]
    for i in range(N)
    for j in range(M)
)

_ = m.addConstrs(
    x_max*(1-wBin[i,j]) - ExprY_ij[i][j] >= w[i,j]
    for i in range(N)
    for j in range(M)
)

In [8]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 384 rows, 444 columns and 1544 nonzeros
Model fingerprint: 0xe8ed09a3
Variable types: 120 continuous, 324 integer (138 binary)
Coefficient statistics:
  Matrix range     [8e-05, 1e+04]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 1.708582e+07
Presolve removed 233 rows and 211 columns
Presolve time: 0.00s
Presolved: 151 rows, 233 columns, 745 nonzeros
Variable types: 23 continuous, 210 integer (50 binary)

Root relaxation: objective 1.073883e+07, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.0739e+07    0   23 1.7086e+07 1.0739e+07  37.1%     -    0s
H    0     0                    1.667621e+07 1.07

In [9]:
for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

x_ijk[0,0,0] -0
x_ijk[0,0,1] -0
x_ijk[0,0,2] -0
x_ijk[0,1,0] -0
x_ijk[0,1,1] -0
x_ijk[0,1,2] -0
x_ijk[0,2,0] -0
x_ijk[0,2,1] -0
x_ijk[0,2,2] 38
x_ijk[0,3,0] -0
x_ijk[0,3,1] -0
x_ijk[0,3,2] -0
x_ijk[0,4,0] -0
x_ijk[0,4,1] -0
x_ijk[0,4,2] -0
x_ijk[0,5,0] -0
x_ijk[0,5,1] -0
x_ijk[0,5,2] -0
x_ijk[1,0,0] -0
x_ijk[1,0,1] -0
x_ijk[1,0,2] -0
x_ijk[1,1,0] -0
x_ijk[1,1,1] -0
x_ijk[1,1,2] -0
x_ijk[1,2,0] -0
x_ijk[1,2,1] -0
x_ijk[1,2,2] 45
x_ijk[1,3,0] -0
x_ijk[1,3,1] -0
x_ijk[1,3,2] -0
x_ijk[1,4,0] -0
x_ijk[1,4,1] -0
x_ijk[1,4,2] -0
x_ijk[1,5,0] -0
x_ijk[1,5,1] -0
x_ijk[1,5,2] -0
x_ijk[2,0,0] -0
x_ijk[2,0,1] -0
x_ijk[2,0,2] -0
x_ijk[2,1,0] -0
x_ijk[2,1,1] -0
x_ijk[2,1,2] 82
x_ijk[2,2,0] -0
x_ijk[2,2,1] -0
x_ijk[2,2,2] 200
x_ijk[2,3,0] -0
x_ijk[2,3,1] -0
x_ijk[2,3,2] -0
x_ijk[2,4,0] -0
x_ijk[2,4,1] -0
x_ijk[2,4,2] -0
x_ijk[2,5,0] -0
x_ijk[2,5,1] -0
x_ijk[2,5,2] -0
x_ijk[3,0,0] -0
x_ijk[3,0,1] -0
x_ijk[3,0,2] -0
x_ijk[3,1,0] -0
x_ijk[3,1,1] -0
x_ijk[3,1,2] 97
x_ijk[3,2,0] -0
x_ijk[3,2,1] -0
x_ijk[3

In [12]:
print(f'purchaseCost: {purchaseCostExpr.getValue()}')
print(f'FixedCost: {FixedCostExpr.getValue()}')
print(f'VariableCost: {VariableCostExpr.getValue()}')
print(f'InventoryCost: {InventoryCostExpr.getValue()}')
print(f'allContainerCost: {allContainerCostExpr.getValue()}')
print(f'AllLostSaleCost: {AllLostSaleCostExpr.getValue()}')

purchaseCost: 15232000.0
FixedCost: 410.0
VariableCost: 6776.0
InventoryCost: 1256400.0
allContainerCost: 13750.0
AllLostSaleCost: 0.0
